# cFos Drug Notebook

## Setup

In [1]:
# Import packages
import os, sys, importlib
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('functionScripts')
sys.path.append('dependencies')

import initFunctions as init #import createDirs, debugReport, loadLightSheetData
import analysisFunctions #compareAnimals, drug_stats_and_changes, brainRegionTrend
import classifyFunctions as cf #sclassifySamples, reformatData
import plotFunctions as pf #totalCountsPlot
import helperFunctions as hf
import configFunctions as config

### Switches

In [2]:
## Switch to control whether Batch 1 and Batch 2 datasets are analyzed together. Distinct folders for each style.
batchSplit = False          # Splits drugs from the first batch of data, from the second, from the 3rd. Batch 1 is labeled with 'a' (aSAL, aKET, aPSI), Batch 3 (cKET, MDMA)
splitTag  = ['a', '', 'c']  # Appended the to beginning of data from the first batch (PSI, KET, SAL -> aPSI, KET, aSAL).

# Debug
debugOutputs = False        # Saves csvs at intervals
scalingFactor = True        # Applies 1/total_cells as a scaling factor for mice in batch one.
debug_ROI = ['Dorsal nucleus raphe']

### Configuration related dictionaries and function calls

In [3]:
switchDir = dict(batchSplit=batchSplit, splitTag=splitTag, debugOutputs=debugOutputs, scalingFactor=scalingFactor, debug_ROI=debug_ROI)

# Make directories, and add their strings to the directory dictionary.
dirDict = init.setPath_createDirs()

# General Figure Settings
config.setup_figure_settings()

# Figure 2 related setup
heatmapDict = config.return_heatmapDict()

#### Figure 3, 4, 5, 6
# Configure the classification - testing for quicker results, default for final.

# classifyDict = config.return_classifyDict_testing()   # Test with this
classifyDict = config.return_classifyDict_default()     # Full run

# Plot dict, used to coordinate outputs of classification plotting.
plotDict = config.return_plotDict()

<Figure size 640x480 with 0 Axes>

### Load/Generate Lightsheet df

In [4]:
# once directories have been made, merge data across batches and with atlases, then save.
lightsheet_data = init.loadLightSheetData(dirDict, switchDir)

# Create the Drug list for reference. Sorting doesn't seem to matter atm.
drugList = list(lightsheet_data.drug.unique())
drugListActive = [x for x in drugList if 'SAL' not in x]

Data being loaded...
Execution time in seconds: 0.0


## Figure 1 - Head twitch response + Cell count per drug

In [ ]:
# pf.plot_headTwitchTotal(dirDict)
# pf.plotTotalPerDrug(lightsheet_data, 'total_cells', dirDict)

## Figure 2 - Data Heatmaps

In [ ]:
# pf.plot_data_heatmap(lightsheet_data, heatmapDict, dirDict)
# pf.distance_matrix(lightsheet_data, classifyDict, dirDict)

## Figure 3 - Drug Classifier

In [ ]:
### Figure 3b - Supervised Dim Red via LDA
ldaDict = dict()
importlib.reload(pf)

pf.plotLowDimEmbed(lightsheet_data, 'count_norm', dirDict, 'LDA', classifyDict, ldaDict) # 'count_norm', 'total_cells','cell_density'

In [ ]:
ldaDict = dict()
importlib.reload(pf)

# ldaDict['AnalysisName'] = [[Training Set], [Testing Set]]
ldaDict['LO_6FDET'] = [['5MEO', 'A-SSRI', 'C-SSRI', 'KET', 'MDMA', 'PSI', 'SAL'], drugList]
ldaDict['LO_6FDET_SSRIs'] = [['5MEO', 'KET', 'MDMA', 'PSI', 'SAL'], drugList]
ldaDict['LO_6FDET_5HT'] = [['PSI', '5MEO'], drugList]

pf.plotLowDimEmbed(lightsheet_data, 'count_norm', dirDict, 'LDA', classifyDict, ldaDict) # 'count_norm', 'total_cells','cell_density'

In [ ]:
### Figure 3c/d - Drug based Confusion Matrix + PR Curves
# importlib.reload(cf)
# importlib.reload(pf)
# importlib.reload(hf)

cf.classifySamples(lightsheet_data, classifyDict, plotDict, dirDict)

## Figure 4 - 'PSI vs' classification

### Run the classifications

In [ ]:
# Revise Plot dict below
plotDict['plot_ConfusionMatrix'] = True
plotDict['plot_PRcurve'] = True
plotDict['plot_SHAPsummary'] = True
plotDict['plot_SHAPforce'] = False
plotDict['featureCorralogram'] = False

# classifyVec = ['class_Psi5MEO', 'class_PsiKet', 'class_PsiMDMA', 'class_PsiDF', 'class_PsiSSRI', 'class_DT', 'class_5HTR', 'class_5HT2A', 'class_SSRI']

classifyVec = []

for i in classifyVec:
    try:
        print(f"Classifying {i}")
        classifyDict['label'] = i
        cf.classifySamples(lightsheet_data, classifyDict, plotDict, dirDict)
    except Exception:
        print(f"\n Failed to classify {i}")

### 4x. Classifications w/o 6-F-DET in training set

In [ ]:
importlib.reload(hf)
importlib.reload(cf)
classifyVec = ['LO_all', 'LO_PSI_5MEO', 'LO_all_nSSRI']
plotDict['plot_ConfusionMatrix'] = True
plotDict['plot_PRcurve'] = False
plotDict['plot_SHAPsummary'] = False
plotDict['plot_SHAPforce'] = False
plotDict['featureCorralogram'] = False
classifyDict['saveLoadswitch'] = True
config.setup_figure_settings()

# classifyDict['label'] = 'LO_all_nSSRI'
# cf.classifySamples(lightsheet_data, classifyDict, plotDict, dirDict)

for i in classifyVec:
    try:
        print(f"Classifying {i}")
        classifyDict['label'] = i
        cf.classifySamples(lightsheet_data, classifyDict, plotDict, dirDict)
    except Exception:
        print(f"\n Failed to classify {i}")

### Figure setup

In [ ]:
importlib.reload(hf)
importlib.reload(config)
importlib.reload(pf)

# Figure Settings
config.setup_figure_settings()

filterByFreq = .75 * classifyDict['CV_count']

sortedNames = ['5MEO vs PSI', 'MDMA vs PSI', 'A-SSRI vs PSI', 'KET vs PSI', '6-F-DET vs PSI', '5MEO vs 6-F-DET', 'A-SSRI vs C-SSRI', 'MDMA vs PSI/5MEO', '6-F-DET vs PSI/5MEO']

sortedNames.reverse()

featureLists, comparisonNames, aucScores, meanScores, aucScrambleScores, meanScrambleScores = hf.retrieve_dict_data(dirDict, classifyDict)

sortIdx = hf.sort_comparison_idx(sortedNames, comparisonNames)

featureLists = [featureLists[i] for i in sortIdx]
comparisonNames = [comparisonNames[i] for i in sortIdx]
aucScores = [aucScores[i] for i in sortIdx]
meanScores = [meanScores[i] for i in sortIdx]
aucScrambleScores = [aucScrambleScores[i] for i in sortIdx]
meanScrambleScores = [meanScrambleScores[i] for i in sortIdx]

### Figure 4b - mean AUPRC across comparisons

In [ ]:
# Color palette
colorsList = [[100, 100, 100], [180, 180, 180]]
colorsList = np.array(colorsList)/256
importlib.reload(pf)

pf.plot_cross_model_AUC(comparisonNames, aucScores, aucScrambleScores, colorsList, dirDict)

### Figure 4c/d - Feature counts and frequencies

In [ ]:
importlib.reload(pf)

# Reverse both only once for the final plot.
# comparisonNames.reverse()
# featureLists.reverse()

pf.plot_featureCount_violin(comparisonNames, featureLists, dirDict)
# pf.plot_similarity_matrix(comparisonNames, featureLists, filterByFreq, dirDict)
# pf.plot_featureOverlap_VennDiagram(comparisonNames, featureLists, filterByFreq, dirDict)
# pf.plot_featureHeatMap(lightsheet_data, comparisonNames, featureLists, filterByFreq, dirDict)9+

## Figure 5/6 - SHAP

### PSI vs 5MEO SHAP analysis

In [ ]:
plotDict['plot_ConfusionMatrix'] = False
plotDict['plot_PRcurve'] = False
plotDict['plot_SHAPsummary'] = True
plotDict['plot_SHAPforce'] = True
importlib.reload(cf)
importlib.reload(pf)

classifyDict['label'] = 'class_Psi5MEO'

cf.classifySamples(lightsheet_data, classifyDict, plotDict, dirDict)

### PSI vs KET, A-SSRI, and MDMA analysis

In [ ]:
# Revise Plot dict below
plotDict['plot_SHAPsummary'] = True
plotDict['plot_SHAPforce'] = False

classifyVec = ['class_PsiMDMA', 'class_PsiSSRI', 'class_PsiKet']

for i in classifyVec:
    try:
        print(f"Classifying {i}")
        classifyDict['label'] = i
        cf.classifySamples(lightsheet_data, classifyDict, plotDict, dirDict)
    except Exception:
        print(f"\n Failed to classify {i}")

# Old Notebook Reintegration

In [5]:
# Additional Processing
data_diff, combined_stats, cfos_diff, cfos_diff_labels = analysisFunctions.drug_stats_and_changes(lightsheet_data, drugList)


In [6]:
# Change figure settings for this large figure
importlib.reload(config)
importlib.reload(pf)
import glob
config.setup_figure_changeFonts(6)

#filter data
cleaned_all_cfos_diff_labels = cfos_diff_labels

#plotting
fileOutName = os.path.join(dirDict['outDir'], 'fig2_deltaPerRegion_SAL')

if len(glob.glob(fileOutName + '.*')) == 0:

    pf.plot_cFos_delta(cfos_diff, drugList, fileOutName)


findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.


## Collecting CIs for Drug - Sal condition

In [ ]:
# Collect the confidence intervals for the prior
if batchSplit:
    # More complicated to focus on aSAL vs SAL for the right drugs.
    drugListA = [x for x in drugList if x[0] == 'a']
    drugListB = [x for x in drugList if x[0] != 'a']
    drugPairListA = [(a + '-'+ b) for idx, a in enumerate(drugListA) for b in drugListA[idx + 1:]]
    drugPairListB = [(a + '-'+ b) for idx, a in enumerate(drugListB) for b in drugListB[idx + 1:]]
    drugPairList = drugPairListA + drugPairListB

    # Brief additional processing to make sure Batch 1 is processed w/ batch 1 saline (aSAL)
    salListA = [i for i in drugPairListA if 'SAL' in i]
    salListB = [i for i in drugPairListB if 'SAL' in i]
    salList = salListA + salListB

else:
    drugPairList = [(a +'-'+ b) for idx, a in enumerate(drugList) for b in drugList[idx + 1:]]
    salList = [i for i in drugPairList if 'SAL' in i]

# Generate some structures to load.
drug_ci_range = np.empty((len(drugListActive),2), dtype=object)
drug_ci_range_drugList = np.empty((len(drugListActive),1), dtype=object)

# Define Paths
tmpFileName = 'drug_ci_range_db_utils.h5.npy'
tempDataFilename = os.path.join(dirDict['tempDir'], tmpFileName)

if os.path.exists(tempDataFilename):
    print('Loading ' + tmpFileName + '...')

    # Load previous saved file
    ciRangeDB = pd.read_pickle(tempDataFilename)

else:
    print('Generating ' + tmpFileName + '...')

    for idx, drugPair in enumerate(tqdm(salList)):
        
        dataInd = list(cleaned_all_cfos_diff_labels).index(drugPair)

        # Make life easier - collect needed dots
        regionList = cfos_diff[dataInd].Region_Name.unique()
        dataCol = cleaned_all_cfos_diff_labels[dataInd]
        dataTable = cfos_diff[dataInd]
        ciData = np.empty([len(regionList), 2])

        # Two methods for determining CIs. Method 2 is faster, and seems to yield identical responses.
        methodSwitch = 2

        if methodSwitch == 1:
        # Method 1 - Sure about region/number match - very slow.
            for region_i, region in enumerate(regionList):
                regionData = dataTable.loc[dataTable.Region_Name == region][dataCol]
                ciData[region_i, :] = sns.utils.ci(sns.algorithms.bootstrap(regionData), which=95)

            if idx == 0:
                ciRangeDB = pd.DataFrame()
                ciRangeDB['Region_Name'] = regionList

            ciRangeDB[drugPair + '_lower'] = ciData[:,0]
            ciRangeDB[drugPair + '_upper'] = ciData[:,1]

        elif methodSwitch == 2:
        # Method 2 - Plot it, things are done faster.
            # Figure
            plt.figure(figsize=(20,160))

            ax = sns.pointplot(y='Region_Name', x=dataCol, data = dataTable, errorbar=('ci', 95), join=False, units=16, errwidth = 0.5, color='red',dodge=True)

            # Pull values
            if idx == 0:
                ciRangeDB = pd.DataFrame()
                ciRangeDB['Region_Name'] = [textObj.get_text() for textObj in ax.axes.get_yticklabels()]

            ciRangeDB[drugPair + '_upper'] = [line.get_xdata().max() for line in ax.lines]
            ciRangeDB[drugPair + '_lower'] = [line.get_xdata().min() for line in ax.lines]
            plt.clf()

    # Save for later use
    ciRangeDB.to_pickle(tempDataFilename)

## mRNA + cFos correlations

In [ ]:
#load data
all_gene_data = pd.read_csv(os.sep.join([dirDict['atlasDir'] , "allGeneStructureInfo_allgenes_summary_struct.csv"]))

#first get mean and sd by gene across whole brain

#clean up a little first

#getting structure averages
all_gene_data_clean = all_gene_data.drop(columns=['Unnamed: 0', 'data_set_id', 'plane_of_section_id'])
StructureAverages = all_gene_data_clean.groupby(['structure_id','gene_acronym']).mean().reset_index()
# StructureAverageDensity = StructureAverage.groupby(['structure_id', 'gene_acronym']).mean().reset_index() # Two tables were identical, double check if done correctly FAQ
# StructureAverages = pd.merge(StructureAverageEnergy, StructureAverageDensity)

#averaging across whole brain
all_gene_data_clean = all_gene_data_clean.drop(columns=['structure_id'])
GeneAverageEnergy = all_gene_data_clean.groupby('gene_acronym').agg({'expression_energy': ['mean', 'std']}).reset_index()
GeneAverageDensity = all_gene_data_clean.groupby('gene_acronym').agg({'expression_density': ['mean', 'std']}).reset_index()
GeneAverages = pd.merge(GeneAverageEnergy, GeneAverageDensity)

#calculate zscore for each brain structure
#fresh copy of structure data
ZscoreStructure = StructureAverages.copy()
ZscoreStructure = ZscoreStructure.merge(GeneAverages, how='left', left_on='gene_acronym', right_on='gene_acronym')

#rename columns for clarity
column_indices = [4,5,6,7]
new_names = ['brain_expression_energy_mean','brain_expression_energy_std','brain_expression_density_mean','brain_expression_density_std']
old_names = ZscoreStructure.columns[column_indices]
ZscoreStructure = ZscoreStructure.rename(columns=dict(zip(old_names, new_names)))

#calculate zscore with structure/gene average and brain wide gene average/std
ZscoreStructure['zscore'] = (ZscoreStructure['expression_energy'] - ZscoreStructure['brain_expression_energy_mean']) / (ZscoreStructure['brain_expression_energy_std'])

In [ ]:
ABA_dict_filt = hf.create_ABA_dict(dirDict)

# get allen names
hierarchy_meso_ids = ABA_dict_filt.rename(columns={'Region ID':'Region_ID'})

#rename/clean for matching
ZscoreStructure = ZscoreStructure.rename(columns={'structure_id':'Region_ID'})
ZscoreStructure = ZscoreStructure.drop(columns=['expression_energy','expression_density','brain_expression_energy_mean','brain_expression_energy_std','brain_expression_density_mean', 'brain_expression_density_std'])

#merge
ZscoreStructure = ZscoreStructure.merge(hierarchy_meso_ids, on='Region_ID', how='inner', suffixes=('', '_x'))
ZscoreStructure = ZscoreStructure.drop(ZscoreStructure.filter(regex='_x$').columns.tolist(),axis=1)

In [ ]:
#mRNA
ZscoreStructure

#cFos
cfos_all = data_diff
# cfos_all = cfos_all.groupby(['Region_Name','abbreviation','Region_ID'])['psi-sal','psi-ket','ket-sal'].mean().reset_index() # Done prior
cfos_all = cfos_all.merge(hierarchy_meso_ids, on='abbreviation')
cfos_all = cfos_all.rename(columns={'Region_ID_x':'Region_ID'})
cfos_all = cfos_all.drop(columns={'Region_ID_y'})

cfos_x_genes = cfos_all.merge(ZscoreStructure, on=['Region_ID','abbreviation','Brain Area'])
cfos_x_genes = cfos_x_genes.rename(columns={'Region_Name_x':'Region_Name'})

#matrices
# cfos_matrix_psi = cfos_x_genes.pivot(values='psi-sal',index='gene_acronym', columns='Region_Name') #psilocybin
# cfos_matrix_ket = cfos_x_genes.pivot(values='ket-sal',index='gene_acronym', columns='Region_Name') #ketamine
gene_matrix = cfos_x_genes.pivot(values='zscore',index='gene_acronym', columns='Region_Name')

In [ ]:
# selected_receptors = ['Htr1a','Htr1b','Htr1d','Htr1f','Htr2a','Htr2b','Htr2c','Htr3a','Htr3b','Htr4','Htr5a','Htr5b','Htr6','Htr7','Slc6a4',
#             'Pvalb', 'Sst', 'Vip', 'Sncg', 'Lamp5',
#             'Gria2', 'Grik2','Grin2b',
#             'Adra2a', 'Adra2b', 'Hrh1', 
#             'Drd1', 'Drd2', 'Drd3', 'Oprm1']

# zscore_select = ZscoreStructure[ZscoreStructure['gene_acronym'].isin(selected_receptors)]
# zscore_select.to_csv(os.path.join(dirDict['outDir'], 'genes_of_interest_whole_brain.csv'))

### All gene x structure correlation (Both brain wide, cortex, and thalamus)

In [ ]:
cfos_x_genes_ctx = cfos_x_genes[cfos_x_genes['Brain Area'] == 'Cortex']
cfos_x_genes_thal = cfos_x_genes[cfos_x_genes['Brain Area'] == 'Thalamus']
regionSet = ['whole', 'thal', 'ctx']

list_of_genes = cfos_x_genes.gene_acronym.unique()

# List of genes shortening for testing loop
# list_of_genes = list_of_genes[0:9]

gene_count = len(list_of_genes)
gene_drug_corr = np.empty([len(drugListActive), len(regionSet)], dtype=object)

drugListActive = ['MDMA', '5MEO', '6-F-DET', 'A-SSRI', 'C-SSRI', 'KET', 'PSI']
# drugListActive = ['MDMA', 'PSI']

for set_i, set_name in enumerate(regionSet):
    for drug_i, drug in enumerate(drugListActive):

        all_correlations = np.empty([gene_count])
        tempDataFilename = os.path.join(dirDict['tempDir'], drug + '_' + set_name + '_corr_db.h5')

        if os.path.exists(tempDataFilename):
            print('Loading  ' + set_name + ' ' + drug + '...')

            # Load previous saved file
            all_correlations = pd.read_pickle(tempDataFilename)
            
        else:
            print('Processing ' + set_name + ' ' + drug + '...')

            if drug[0] == splitTag[0]:
                drugCol = drug + '-' + 'aSAL'
            else:
                drugCol = drug + '-SAL'

            # Based on the desired comparison, use different datasets
            if set_name == 'ctx':
                cfos_data_genes = cfos_x_genes_ctx.copy()
            elif set_name == 'thal':
                cfos_data_genes = cfos_x_genes_thal.copy()
            elif set_name == 'whole':
                cfos_data_genes = cfos_x_genes.copy()
            else:
                raise ValueError('Invalid region set name.')

            # Iterate across genes, finding correlations
            for gene_i, gene in enumerate(tqdm(list_of_genes)):
                cfos_x_genes_of_interest = cfos_data_genes[cfos_data_genes['gene_acronym'] == gene]
                correlation = cfos_x_genes_of_interest['zscore'].corr(cfos_x_genes_of_interest[drugCol])
                all_correlations[gene_i] = correlation
            
            all_correlations = pd.DataFrame(np.column_stack((list_of_genes, all_correlations)), columns=['gene', drug + ' correlation'])

            # Save for later use
            all_correlations.to_pick le(tempDataFilename)  
            
        gene_drug_corr[drug_i, set_i] = all_correlations


In [ ]:
#list genes of interest for correlation
# geneList = ['Htr1a','Htr1b','Htr1d','Htr1f','Htr2a','Htr2b','Htr2c','Htr3a','Htr3b','Htr4','Htr5a','Htr5b','Htr6','Htr7','Slc6a4',
#             'Pvalb', 'Sst', 'Vip', 'Sncg', 'Lamp5', 'Slc17a7', 'Plxnd1', 'Fezf2', 'Gad1', 'Camk2a', 
#             'Gria2', 'Grik2','Grin2b',
#             'Adra2a', 'Adra2b', 'Hrh1', 
#             'Drd1', 'Drd2', 'Drd3', 'Oprm1']

geneList = ['Htr1a','Htr1b','Htr1d','Htr2a','Htr2b','Htr2c','Htr1f', 'Htr3a','Htr3b','Htr4','Htr5a','Htr5b','Htr6','Htr7','Slc6a4', 
            'Pvalb', 'Sst', 'Vip', 'Sncg', 'Lamp5', 'Slc17a7', 'Plxnd1', 'Fezf2', 'Gad1', 'Camk2a', 
            'Grin1', 'Grin2a', 'Grin2b', 'Grin2c', 'Grin2d', 'Gria1', 'Gria2', 'Gria3', 'Gria4', 'Grin2b', 'Grik2', 'hcn1',
            'Hrh1',
            'Slc6a2', 'Slc6a3', 'Slc6a4', 'Adra1a', 'Adra1d', 'Adrb1', 'Adrb2', 'Adra2a', 'Adra2b', 'Adra2c', 'Adrbk1', 'Adrbk2', 'Drd1', 'Drd2', 'Drd3', 'Drd4', 'Drd5'] 


for set_i, set in enumerate(regionSet):
    for drug_i, drug in enumerate(drugListActive):

        drug_data = gene_drug_corr[drug_i, set_i]
        
        #Sort the correlations for each drug
        colData = drug_data[drug + ' correlation']
        drug_data['percentile'] = colData.rank(pct = True)
        drug_data = drug_data.sort_values(by=['percentile'], ascending=False)

        gene_drug_corr[drug_i, set_i] = drug_data

        genes_of_interest = drug_data[drug_data['gene'].isin(geneList)]
        genes_of_interest.to_csv(os.path.join(dirDict['outDir'], drug + '_' + set + '_genes_of_interest.csv'))

### Cortex x Receptors

In [ ]:

exportFormat = 'png' # can also be 'svg' 
plotNameDict = {'KET': 'Ketamine', 'aKET': 'Ketamine (B1)', 'cKET': 'Ketamine (B3)', 'PSI': 'Psilocybin', 'aPSI': 'Psilocybin (B1)', 
                'A-SSRI': 'Acute SSRI', 'C-SSRI': 'Chronic SSRI', 
                '5MEO': '5-MeO-Dimethyltryptamine (5MEO)', '6-F-DET': '6-Flouro-Diethyltryptamine (6-F-DET)', 'MDMA': '3,4 - MDMA'}
    
receptorPlotList = ['Htr1a','Htr1b','Htr1d','Htr2a','Htr2b','Htr2c','Htr1f', 'Htr3a','Htr3b','Htr4','Htr5a','Htr5b','Htr6','Htr7','Slc6a4', # - For Serotonin related - PSI, 5MEO, 6FDET, A-SSRI, C-SSRI
            #'Pvalb', 'Sst', 'Vip', 'Sncg', 'Lamp5', 'Slc17a7', 'Plxnd1', 'Fezf2', 'Gad1', 'Camk2a', 
            'Grin1', 'Grin2a', 'Grin2b', 'Grin2c', 'Grin2d', 'Gria1', 'Gria2', 'Gria3', 'Gria4', 'Grin2b', 'Grik2', 'hcn1' # - For NMDA receptor, Ketamine (binds to Grin1), some HCN1 impact ('Receptors and beyond')
            #'Adra2a', 'Adra2b', 'Hrh1', 
            'Slc6a2', 'Slc6a3', 'Slc6a4', 'Adra1a', 'Adra1d', 'Adrb1', 'Adrb2', 'Adra2a', 'Adra2b', 'Adra2c', 'Adrbk1', 'Adrbk2', 'Drd1', 'Drd2', 'Drd3', 'Drd4', 'Drd5'] # - DA, 5HT, and NE transporters and receptors

# MDMA - Norepinephrine, Dopamine, Serotonin Transporter + All their receptors?
# *** Question - Inhibitory receptors - you'd expect something BELOW baseline, right?
# Negative effect? " while 5-HT2A receptor densities were significantly reduced in all cortical brain regions studied in recent MDMA users when compared with control subjects. "

sertGenes = ['Htr1a','Htr1b','Htr1d','Htr2a','Htr2b','Htr2c','Htr1f', 'Htr3a','Htr3b','Htr4','Htr5a','Htr5b','Htr6','Htr7']
dopaGenes = ['Grin1', 'Grin2a', 'Grin2b', 'Grin2c', 'Grin2d', 'Gria1', 'Gria2', 'Gria3', 'Gria4', 'Grik2', 'Hcn1']
sertTransGene = ['Slc6a2', 'Slc6a3', 'Slc6a4', 'Adra1a', 'Adra1d', 'Adrb1', 'Adrb2', 'Adra2a', 'Adra2b', 'Adra2c', 'Adrbk1', 'Adrbk2', 'Drd1', 'Drd2', 'Drd3', 'Drd4', 'Drd5']

receptorPlotList = np.concatenate([sertGenes, dopaGenes, sertTransGene])

# Craft the color range - Index in array used to sample colors.
htrInd = 1
GriaInd = 70
DrdInd = 200
cStep = 5

import numpy as np
sertColors = np.arange(0, cStep*len(sertGenes), cStep) + htrInd
dopaColors = np.arange(0, cStep*len(dopaGenes), cStep) + GriaInd
sertTColors = np.arange(0, cStep*len(sertTransGene), cStep) + DrdInd

receptorPlotColors = np.concatenate([sertColors, dopaColors, sertTColors])

# Dopamine
# Whole brain, Cortex, Thalamus
# DAT, SERT, Glut, Metabotropic Glutamate Receptors (mGluRs).

genePlotColorPalette = sns.color_palette("Spectral", as_cmap=True, n_colors=len(receptorPlotList))
regionSetDict = {'whole': 'Whole Brain', 'ctx': 'Cortex', 'thal': 'Thalamus'}

cellTypePlotList = ['Pvalb', 'Sst', 'Vip', 'Lamp5', 'Slc17a7', 'Plxnd1', 'Gad1', 'Fezf2']
cellTypePlotColors = list(np.arange(0, len(cellTypePlotList)*10, 10))

# Create structure to loop over. Receptor vs Cell Type gene lists
geneSets = [receptorPlotList, cellTypePlotList]                
geneSetTitle = ['Receptor', 'Cell Type']          
geneSetColorInd = [receptorPlotColors, cellTypePlotColors]

for gene_set_i, gene_set in enumerate(geneSets):

    genePlotList = gene_set
    genePlotColors = geneSetColorInd[gene_set_i]

    for set_i, set_name in enumerate(regionSet):
        for drug_i, drug in enumerate(drugListActive):
            
            if drug[-1] == 't':
                break

            print(drug)
            
            #plot the distribution for all the gene correlations
            plt.figure(figsize=(45,15))
            sns.set(style="ticks", font_scale=8)
            drug_data = gene_drug_corr[drug_i, set_i]

            ax = sns.histplot(data=drug_data , x = drug + " correlation", element = 'step', fill = False, color='grey', lw=7)
            sns.despine()

            trans = ax.get_xaxis_transform()

            genes_of_interest = drug_data[drug_data['gene'].isin(genePlotList)]
            corrData = list(genes_of_interest[drug + ' correlation'])

            idx = {x:i for i,x in enumerate(gene_set)}
            idx2 = [idx[x] for x in genes_of_interest['gene'] if x in idx]  
            genes_of_interest['index'] = idx2
            genes_of_interest = genes_of_interest.sort_values('index')

            genes_of_interest['colorInd'] = genePlotColors
            genes_of_interest = genes_of_interest.sort_values(drug + ' correlation')

            # Generate spots for the text above the plots to go. Move text over if it is overlapping with text to the left
        
            textXVals = np.array(genes_of_interest[drug + ' correlation'])
            minDist = .03

            # for xval_i in range(0, len(textXVals)-1):
            #     if abs(textXVals[xval_i] - textXVals[xval_i+1]) < minDist:
            #         midPoint = np.mean(textXVals[xval_i:xval_i+2])
            #         textXVals[xval_i] = midPoint - minDist/2
            #         textXVals[xval_i+1] = midPoint + minDist/2

            # genes_of_interest['textPlotX'] = textXVals

            xLimits = ax.get_xlim()
            xLimLeftLabel = textXVals[0] #xLimits[0]+(xLimits[1]*.3)
            if abs(textXVals[0]-textXVals[-1]) < 0.2:
                xLimRightLabel = textXVals[-1] + .2
            else:
                xLimRightLabel = textXVals[-1] #xLimits[0]+(xLimits[1]*.7)
            textXAxes = np.linspace(xLimLeftLabel, xLimRightLabel, num:=len(receptorPlotList))

            for gene_i, gene in enumerate(genes_of_interest.gene):
                
                geneData = genes_of_interest[genes_of_interest.gene == gene]
                corrVal = float(geneData[drug + ' correlation'])
                prcVal= round(float(geneData['percentile'])*100)
                lineCol = genePlotColorPalette(geneData.colorInd)[0]

                plt.axvline(x=corrVal, color=lineCol, linestyle='--', lw=10)
                plt.text(textXAxes[gene_i], 1.02, gene + '('+ str(prcVal) + '%)', transform=trans, color=lineCol, rotation=45, fontsize=60.0)
            
            plotLineWidth = 5

            ax.spines['left'].set_linewidth(plotLineWidth)
            ax.spines['bottom'].set_linewidth(plotLineWidth)
            ax.xaxis.set_tick_params(length=40, width=plotLineWidth)
            ax.yaxis.set_tick_params(length=40, width=plotLineWidth)

            ax.spines['left'].set_linewidth(plotLineWidth)
            ax.spines['bottom'].set_linewidth(plotLineWidth)
            ax.xaxis.set_tick_params(length=40, width=plotLineWidth)
            ax.yaxis.set_tick_params(length=40, width=plotLineWidth)
            plt.xlabel('Correlation')
            plt.ylabel('Number of genes')

            plotTitle = plotNameDict[drug] + ' vs. ' + geneSetTitle[gene_set_i] + ' (' + regionSetDict[set_name] + ')'

            plt.title(plotTitle, loc='center',  pad=300)

            plt.savefig(os.path.join(dirDict['outDir'], plotTitle + '.svg'), bbox_inches='tight')
            plt.show()
            plt.clf()